In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
spark.conf.set(
    "fs.azure.account.key.ksrdatadlsa.dfs.core.windows.net",
    dbutils.secrets.get(scope="optumscope1", key="optumkeysstore"))

In [0]:
Subscriber = spark.read.csv("abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/RawData/subscriber.csv",header=True, inferSchema=True,escape='"')

In [0]:
# drop duplicates
Subscriber = Subscriber.dropDuplicates()

In [0]:
# to find Rows count and Cloumn Count
Subscriber.count(),len(Subscriber.columns)

In [0]:
from pyspark.sql.functions import col, isnan, asc

def analyze_columns(df):
    columns = df.columns
    for column in columns:
        distinct_count = df.select(column).distinct().count()
        numeric_count = df.filter(col(column).rlike("^[0-9]")).count()
        text_count = df.filter(col(column).rlike("^[A-Za-z]")).count()
        special_char_count = df.filter(~col(column).rlike("^[A-Za-z0-9]")).count()
        contains_number = df.filter(col(column).rlike("[0-9]")).count()
        contains_string = df.filter(col(column).rlike("[A-Za-z]")).count()
        contains_spl_char = df.filter(col(column).rlike("[^A-Za-z0-9]")).count()

        print("Distinct Count of {} column: {}".format(column, distinct_count))
        print("{} Column Start With Number: {}".format(column, numeric_count))
        print("{} Column Start With String: {}".format(column, text_count))
        print("{} Column Start With Spl.Cha: {}".format(column, special_char_count))
         print("{} Column contains number values: {}".format(column, contains_number))
        print("{} Column contains string values: {}".format(column, contains_string))
        print("{} Column contains Spl.Char values: {}\n".format(column, contains_spl_char))
        
# Example usage:
analyze_columns(Subscriber)


In [0]:
Subscriber = Subscriber.fillna("Guest", subset=["first_name"])


In [0]:
Subscriber.filter(col("first_name").isNull()).count()

0

In [0]:
Subscriber.filter(col("first_name").isNaN()).count()

0

In [0]:
print(" Null Count: " ,Subscriber.filter(col("Phone").isNull()).count())
print(" Null Count: " ,Subscriber.filter(col("Phone").isNaN()).count())
print(Subscriber.filter(col("Phone").isNull()).show())

 Null Count:  3
 Null Count:  0
+----------+----------+---------+-----------+----------+------+-----+-------+----------------+--------+---------+--------+----------+----------+
|    sub_id|first_name|last_name|     Street|Birth_date|Gender|Phone|Country|            City|Zip Code|Subgrp_id|Elig_ind|  eff_date| term_date|
+----------+----------+---------+-----------+----------+------+-----+-------+----------------+--------+---------+--------+----------+----------+
| SUBID1036|   Upasana|   Thakur|  Vasa Ganj|1927-10-03|Female| NULL|  India|          Ratlam|  326733|     S108|       N|1947-10-03|1951-02-20|
| SUBID1041|          |   Rajput| Sinha Path|1930-11-25|Female| NULL|  India|        Agartala|  303503|     S110|       Y|1950-11-25|1957-09-14|
|SUBID10095|   Ekaaksh|      Rai|Bansal Ganj|1933-12-02|Others| NULL|  India|Pimpri-Chinchwad|  158186|     S107|       N|1953-12-02|1960-07-29|
+----------+----------+---------+-----------+----------+------+-----+-------+----------------+----

In [0]:
Subscriber = Subscriber.fillna("NA", subset=["Phone"])

In [0]:
# To find null values
from pyspark.sql.functions import count, col, isnan, when
Subscriber.select([count(when(col(c).isNull(), c)).alias(c) for c in Subscriber.columns]).show()

0

In [0]:
from pyspark.sql.functions import length
Subscriber.filter(length(col("Zip Code")) == 6).count()


93

In [0]:
from pyspark.sql.functions import length
Subscriber.filter(length(col("Zip Code")) < 6).count()

7

In [0]:
from pyspark.sql.functions import length
Subscriber.filter(length(col("Zip Code")) > 6).count()

0

In [0]:
from pyspark.sql.functions import when

# Define replacement conditions using when function
conditions = [
    (col("city") == "Tiruvottiyur", 600019),
    (col("city") == "Bihar Sharif", 803118),
    (col("city") == "Hapur", 245101),
    (col("city") == "Kochi", 682001),
    (col("city") == "Hajipur", 844101),
    (col("city") == "Ghaziabad", 201003),
    (col("city") == "Navi Mumbai", 400001)
]

# Apply the conditions using otherwise function
Subscriber = Subscriber.withColumn("Zip Code", when(conditions[0][0], conditions[0][1])
                                                  .when(conditions[1][0], conditions[1][1])
                                                  .when(conditions[2][0], conditions[2][1])
                                                  .when(conditions[3][0], conditions[3][1])
                                                  .when(conditions[4][0], conditions[4][1])
                                                  .when(conditions[5][0], conditions[5][1])
                                                  .when(conditions[6][0], conditions[6][1])
                                                  .otherwise(col("Zip Code")))


In [0]:
# To find null values
from pyspark.sql.functions import count, col, isnan, when
Subscriber.select([count(when(col(c).isNull(), c)).alias(c) for c in Subscriber.columns]).show()

In [0]:
from pyspark.sql.functions import when

Subscriber = Subscriber.withColumn("Subgrp_id", 
                    when(Subscriber["SUB_ID"] == "SUBID10022", "S110")
                   .when(Subscriber["SUB_ID"] == "SUBID10049", "S107")
                   .otherwise(Subscriber["Subgrp_id"]))

In [0]:
Subscriber.filter(col("Subgrp_id").isNull()).show()

+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+
|sub_id|first_name|last_name|Street|Birth_date|Gender|Phone|Country|City|Zip Code|Subgrp_id|Elig_ind|eff_date|term_date|
+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+
+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+



In [0]:
Subscriber.filter(col("Elig_ind").isNull()).count()

4

In [0]:
Subscriber = Subscriber_TB.fillna("N", subset="Elig_ind")

In [0]:
Subscriber.filter(col("Elig_ind").isNull()).count()

0

In [0]:
from pyspark.sql.functions import year, current_date, to_date
# Convert the 'patient_birth_date' column to a DateType
Subscriber = Subscriber.withColumn("Birth_date", to_date("Birth_date"))
# Calculate the age based on the birth date
Subscriber = Subscriber.withColumn("age", year(current_date()) - year("Birth_date"))

In [0]:
Subscriber = Subscriber.drop(col("Birth_date"))

In [0]:
Subscriber.select("*").show()

In [0]:
# Define the output container path
output_container_path = "abfss://optumdata@ksrdatadlsa.dfs.core.windows.net/StatgingData"

# Write the DataFrame to the output container path
Subscriber.coalesce(1).write.mode("overwrite").option("header", "true").format("com.databricks.spark.csv").save(output_container_path)

# List files in the output container path
files = dbutils.fs.ls(output_container_path)

# Filter out the output file
output_file = [x for x in files if x.name.startswith("part-")]

# Move the output file to a specific location
dbutils.fs.mv(output_file[0].path, "%s/stg_Subscriber.csv" % output_container_path)